In [ ]:
# Package Imports - feel free to add what you think might be useful! 
import torch
import tensorflow as tf
import numpy as np 
import matplotlib.pyplot as plt 

## Task 1: Playing with Pseudo-Randomness

### Part A: Implement LCG

In [ ]:
def Linear_Congruential_Generator(seed, multipler, increment, modulus):
    sequence = []
    pass # TODO: implement the LCG here!
    return sequence 

In [ ]:
pass # TODO: run your generator for at least 10e5 steps

In [ ]:
pass # TODO: plot a histogram of the generated numbers to visualize distribution

In [ ]:
pass # TODO: ensure at least one parameter set satisfies the Hull-Dobell Theorem and demonstrates a full-period LCG

### Part B: Randomness over Samples

In [ ]:
def ComputeEntropy(X):
    """ 
    You can pass in the random variable X in whatever form you choose.
    One way to represent a random variable is as a dictionary, 
    where each (key, value) pair represents an outcome x 
    and its probability P(X = x)

    Hint: NumPy might be helpful here!
    """
    pass # TODO: copmute Shannon Entropy or Min Entropy

In [ ]:
""" 
1. fix a short seed value for each LCG 
(no more than 2^10 possible values for the seed)

2. Over a long LCG sequence (at least 10^6 samples),
as the generator produces values, treat the sequence of outputs 
up to each step as a growing empirical distribution.

3. At regular intervals (e.g., every 1000 iterations), 
compute the Shannon Entropy or Min-Entropy 
of the current sample distribution.

Hint: We've already imported matplotlib for you, 
though any plotting package will do the trick.
"""
pass # TODO: Plot how the Shannon Entropy or Min-Entropy evolves with the number of iterations for each LCG configuration.

### Part C: Periodicity and Autocorrelation

In [ ]:
pass # TODO: Similarly, using your previous LCGs generate a long LCG sequence (at least 10^6 samples).

In [ ]:
pass # TODO: For each sequence compute the period and compute the autocorrelation over multiple lags.

In [ ]:
pass # TODO: Provide line plots of autocorrelation rho_k against the lag k 

### Part D: Breaking the Illusion of Randomness

In [ ]:
pass # TODO: Apply statistical tests: either chi-squared or Kolmogov-Smirnov to test if your generated random number sequences are random by comparing them to uniform distributions.

### Bonus Task:
Train a light-weight model (small-neural net or decision tree) to predict the next number in your random number sequence. Now try implementing at least two other more robust PRNGs from the list below:
- XOR-Shift Generator
- Permuted Congruential Generator (PCG) 
- SplitMix64 

In [ ]:
# You may want to import these packages, though feel free to import anything else you need
import torch 
import torch.nn as nn 
import torch.nn.functional as F 

import tensorflow as tf 
import keras 

import sklearn